In [28]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

#load the spaCy model
nlp = spacy.load("en_core_web_sm")

#Import file and into excel fle
file_path = 'C:/Users/rgae/OneDrive - QuidelOrtho/Documents/All QuidelFiles/Excel Files/6-21-2024 Copy of CTS.xlsx'
data = pd.read_excel(file_path, index_col=None, na_values=['NA'], usecols='BN, BM')
data['combined_text'] = data.astype(str).agg(' '.join, axis=1)
df = pd.DataFrame(data)

#Tokenize the text using spaCy and create a list of sentences
def tokenize(text):
    doc = nlp(text.lower())
    return ' '.join([token.text for token in doc]) #https://stackoverflow.com/questions/57187116/how-to-modify-spacy-tokens-doc-doc-tokens-with-pipeline-components-in-spacy

df['tokenized_text'] = df['combined_text'].apply(tokenize) #call the method above to the combined text column 
#cant do df['tokenized_text'] = tokenize(...) because you can't input series into a string parameter. Utilizing apply allows us to get around this and apply it for each row

#Use CountVectorizer to get common phrases
vectorizer = CountVectorizer(ngram_range = (1, 5))  #Bigrams and trigrams (how long each phrase can be i.e. 2 and 3)
X = vectorizer.fit_transform(df['tokenized_text'])

#Sum up the counts of each phrase
phrase_counts = X.sum(axis=0).A1
phrases = vectorizer.get_feature_names_out()

#Create a DataFrame with phrases and their counts
phrase_counts_df = pd.DataFrame({'Phrase': phrases, 'Count': phrase_counts}).sort_values(by='Count', ascending=False).reset_index()
phrase_counts_df = phrase_counts_df.drop(['index'], axis = 1)

phrase_counts_df.to_json('phrase.json')

,Phrase,Count
0,the,2092
1,to,1174
2,blade,1170
3,and,1053
4,dispense,941
...,...,...
55111,fallen out it,1
55112,fallen out it is,1
55113,fallen out of assembly,1
55114,fallen out of the,1


In [35]:
import pandas as pd
import numpy as np
import spacy
from collections import Counter

nlp = spacy.load("en_core_web_sm")

file_path = 'C:/Users/rgae/OneDrive - QuidelOrtho/Documents/All QuidelFiles/Excel Files/6-21-2024 Copy of CTS.xlsx'
#put he excel file through a dataframe
df_view = pd.read_excel(file_path, index_col = None, na_values = ['NA'], usecols = 'BN, BM')

#create a combined column to find the frequency
df_view['combined_text'] = df_view.astype(str).agg(' '.join, axis=1)
combined_text = ' '.join(df_view['combined_text'])

#Process the combined text with spaCy
doc = nlp(combined_text)

#Extract words and calculate their frequencies
words = [token.text.lower() for token in doc if token.is_alpha]
word_freq = Counter(words)

#convert the frequency data to a DataFrame for better visualization
word_freq_df = pd.DataFrame(word_freq.items(), columns=['Word', 'Frequency']).sort_values(by='Frequency', ascending=False).reset_index()
word_freq_df = word_freq_df.drop(columns = ['index'])
word_freq_df = word_freq_df.drop(word_freq_df[word_freq_df['Frequency'] <= 25].index)

display(word_freq_df)

,Word,Frequency
0,the,2092
1,to,1174
2,blade,1141
3,and,1053
4,dispense,939
...,...,...
227,both,27
228,telephone,27
229,an,26
230,remove,26


In [14]:
import pandas as pd
import plotly.express as px
from plotly import graph_objects as go

#File path to your Excel file
file_path = 'C:/Users/rgae/OneDrive - QuidelOrtho/Documents/All QuidelFiles/Excel Files/6-21-2024 Copy of CTS.xlsx'

#read the Excel file and extract necessary columns
df = pd.read_excel(file_path, index_col=None, na_values=['NA'], usecols='AL, BN, BM')

#combine the text columns into a single text column for analysis
df['combined_text'] = df.astype(str).agg(' '.join, axis=1)

#define the function to count keyword frequencies
def count_keywords(text, keywords):
    text = text.lower()
    return {keyword: 1 if keyword in text else 0 for keyword in keywords} #binary values

#initialize what keywords you want here, LOWER CASE ONLY
keywords = ['dust', 'dispense blade', 'debris']

#Apply the function to each row in the DataFrame
df['keyword_frequency'] = df['combined_text'].apply(lambda x: count_keywords(x, keywords))

final_frequency = {}
#Sum the values of the entire column by keyword
for keyword in keywords:
    total_sum = {keyword: sum(df['keyword_frequency'].apply(lambda x: x[keyword]))}
    final_frequency.update(total_sum)

#get date dataframe
df['date'] = pd.to_datetime(df['Complete Loc Dt'])

#group dates with the words
keyword_date_frequency = df.groupby('date')['keyword_frequency'].apply(lambda x: pd.DataFrame(list(x)).sum()).reset_index()
keyword_date_frequency = keyword_date_frequency.rename(columns = {'level_1' : 'Keyword', 'keyword_frequency': 'Service Calls', 'date': 'Date'})

#histogram side to side (https://plotly.com/python/bar-charts/)
#Change to graph object and send over next time
fig = px.histogram(keyword_date_frequency, x = 'Date', y = 'Service Calls', color = 'Keyword', barmode = 'group', title = 'Keyword Presence Over Time')
fig.show()

#not working quite well here
fig1 = go.Figure(
    data = [
        go.Bar(name = keywords[0],
               x = keyword_date_frequency['Date'],
               y = keyword_date_frequency['Service Calls'])
    ],
    layout = go.Layout(
        title = 'Keyword Presence Over Time',
        yaxis_title = 'Service Calls',
        xaxis_title = 'Date',
        barmode = 'group'
    )
)
#fig1.show()


In [38]:
import pandas as pd
import plotly.express as px
from plotly import graph_objects as go
import ipywidgets as widgets
from IPython.display import display

import numpy as np
import spacy
from collections import Counter

import dash
from dash import dcc, html
from dash.dependencies import Input, Output

import dash_bootstrap_components as dbc #used for formatting layout of the dash

nlp = spacy.load("en_core_web_sm")

#File path to your Excel file
file_path = 'C:/Users/rgae/OneDrive - QuidelOrtho/Documents/All QuidelFiles/Excel Files/6-21-2024 Copy of CTS.xlsx'
#Read the Excel file and extract necessary columns
df = pd.read_excel(file_path, index_col = None, na_values = ['NA'], usecols = 'AL, BN, BM')

#use pd to date time to utilize in the mapping of the data over time
df['Complete Loc Dt'] = pd.to_datetime(df['Complete Loc Dt'])
#Combine the text columns into a single text column for analysis
df['combined_text'] = df.astype(str).agg(' '.join, axis=1)



#find frequency of text
combined_text = ' '.join(df['combined_text'])
doc = nlp(combined_text)

#Extract words and calculate their frequencies
words = [token.text.lower() for token in doc if token.is_alpha]
word_freq = Counter(words)

#convert the frequency data to a DataFrame for better visualization
word_freq_df = pd.DataFrame(word_freq.items(), columns=['Word', 'Frequency']).sort_values(by='Frequency', ascending=False).reset_index()

words_to_filter = ['the', 'be', 'to', 'of', 'and', 'a', 'in', 'that', 'I', 'i']
word_freq_df = word_freq_df[~word_freq_df['Word'].isin(words_to_filter)]

#Define the function to count keyword frequencies
def count_keywords(text, keywords):
    text = text.lower()
    return {keyword: 1 if keyword in text else 0 for keyword in keywords} #binary values
#############################

#initialize app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.CYBORG])
app.layout = html.Div([  # dbc rows and col where number of columns and rows are determined by how many rows and columns are in the parameters
    dbc.Row(  # e.g. row(col col col) <-- 3 columns || row (col) <-- 1 column https://dash-bootstrap-components.opensource.faculty.ai/docs/components/layout/
        [
            dbc.Col([
                html.H2("Keyword Analysis", style={'font-size': '20px'}),  # H2 indicates sub heading with the following properties
                dcc.Dropdown(
                    id = 'keyword-dropdown',
                    options = [{'label': f"{word} ----- {freq}", 'value': word} for word, freq in zip(word_freq_df['Word'], word_freq_df['Frequency'])],
                    multi = True,
                    placeholder = 'Select keywords',
                    value = [],
                    style = {'width': '100%'}
                ),
                dbc.Button(
                    id = 'submit-button',
                    n_clicks = 0,
                    children = 'Submit',
                    color = 'primary',
                    style = {'margin-top': '10px'}
                ),
            ], width = {'size': '10'},
                style = {'font-size': '16px'}),

            dbc.Col([
                html.H2("Toggle Stack", style={'font-size': '20px'}),
                dcc.Checklist(
                    id = 'toggle-checklist',
                    options = [
                        {'label': 'Grouped', 'value': 'group'},
                    ], value = ['True'],
                    style = {'font-size': '16px'},
                )
            ])
        ], justify = 'between'
    ),

    dbc.Row([
        dbc.Col([
            dcc.Graph(id='keyword-graph')  # Populate graph based on the dropdown selection
        ]),
    ]),
])


#set up an callback function that updates the output based on our input
@app.callback(
    Output(component_id = 'keyword-graph', component_property = 'figure'),
    [Input(component_id = 'submit-button', component_property = 'n_clicks'),
     Input(component_id = 'toggle-checklist', component_property = 'value')],
    [dash.dependencies.State('keyword-dropdown', 'value')]
)

#create function that updates figure
def update_graph(n_clicks, toggle, keywords):  
    #want it so that for each click, updates using property of n_clicks and changing the keywords input

    #consider the case that exist empty string
    if not keywords:
        #return empty dict if exist empty string
        return {}

    #apply the function here which applies strip() and lower() while splitting by ','
    df['keyword_frequency'] = df['combined_text'].apply(lambda x: count_keywords(x, keywords))

    #convert back to dataframe and fill zero if missing(N/A)
    keyword_df = df['keyword_frequency'].apply(pd.Series).fillna(0)

    #group by date and reset the index like before
    keyword_bydate_df = df[['Complete Loc Dt']].join(keyword_df).groupby('Complete Loc Dt').sum().reset_index()
    keyword_bydate_df = keyword_bydate_df.melt(id_vars=['Complete Loc Dt'], var_name='Keyword', value_name='Service Calls')
    keyword_bydate_df = keyword_bydate_df.rename(columns = {'Complete Loc Dt': 'Date'})

    #set the barmode based on the toggle value
    barmode = 'group' if 'group' in toggle else 'stack'

    #create the histogram here
    fig = px.histogram(keyword_bydate_df, x = 'Date', y = 'Service Calls', color = 'Keyword', barmode = barmode, title = 'Keyword Presence Over Time')
    fig.update_xaxes(
        dtick=86400000.0 * 14 , #biweekly
        #tickformat="%b\n%Y",
        ticklabelmode="period"
    )

    #return the fig
    return fig
    
    
#############################

if __name__ == '__main__':
    app.run_server(debug = True, port = 8051) #specify port, couldn't terminate port 8050 so work around for now
    

In [43]:
import pandas as pd
import plotly.express as px
from plotly import graph_objects as go
import ipywidgets as widgets
from IPython.display import display

import numpy as np
import spacy
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

import dash
from dash import dcc, html
from dash.dependencies import Input, Output

import dash_bootstrap_components as dbc #used for formatting layout of the dash

nlp = spacy.load("en_core_web_sm")

#File path to your Excel file
file_path = 'C:/Users/rgae/OneDrive - QuidelOrtho/Documents/All QuidelFiles/Excel Files/6-21-2024 Copy of CTS.xlsx'
#Read the Excel file and extract necessary columns
df = pd.read_excel(file_path, index_col = None, na_values = ['NA'], usecols = 'AL, BN, BM')

#use pd to date time to utilize in the mapping of the data over time
df['Complete Loc Dt'] = pd.to_datetime(df['Complete Loc Dt'])
#Combine the text columns into a single text column for analysis
df['combined_text'] = df.astype(str).agg(' '.join, axis=1)

#Tokenize the text using spaCy and create a list of sentences
def tokenize(text):
    doc = nlp(text.lower())
    return ' '.join([token.text for token in doc]) #https://stackoverflow.com/questions/57187116/how-to-modify-spacy-tokens-doc-doc-tokens-with-pipeline-components-in-spacy

df['tokenized_text'] = df['combined_text'].apply(tokenize) #call the method above to the combined text column 
#cant do df['tokenized_text'] = tokenize(...) because you can't input series into a string parameter. Utilizing apply allows us to get around this and apply it for each row



#Use CountVectorizer to get common phrases
vectorizer = CountVectorizer(ngram_range = (1, 4))  #Bigrams and trigrams (how long each phrase can be i.e. 2 and 3)
X = vectorizer.fit_transform(df['tokenized_text'])

#Sum up the counts of each phrase
phrase_counts = X.sum(axis=0).A1
phrases = vectorizer.get_feature_names_out()

#Create a DataFrame with phrases and their counts
phrase_counts_df = pd.DataFrame({'Phrase': phrases, 'Count': phrase_counts}).sort_values(by = 'Count', ascending = False).reset_index()
phrase_counts_df = phrase_counts_df.drop(['index'], axis = 1)

words_to_filter = ['the', 'be', 'to', 'of', 'and', 'a', 'in', 'that', 'I', 'i']
phrase_counts_df = phrase_counts_df[~phrase_counts_df['Phrase'].isin(words_to_filter)]
phrase_counts_df = phrase_counts_df.drop(phrase_counts_df[phrase_counts_df['Count'] <= 1].index)


#Define the function to count keyword frequencies
def count_keywords(text, keywords):
    text = text.lower()
    return {keyword: 1 if keyword in text else 0 for keyword in keywords} #binary values
#############################

#initialize app
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.CYBORG])
app.layout = html.Div([  # dbc rows and col where number of columns and rows are determined by how many rows and columns are in the parameters
    dbc.Row(  # e.g. row(col col col) <-- 3 columns || row (col) <-- 1 column https://dash-bootstrap-components.opensource.faculty.ai/docs/components/layout/
        [
            dbc.Col([
                html.H2("Phrase Analysis", style={'font-size': '20px'}),  # H2 indicates sub heading with the following properties
                dcc.Dropdown(
                    id = 'keyword-dropdown',
                    options = [{'label': f"{phrase} ----- {freq}", 'value': phrase} for phrase, freq in zip(phrase_counts_df['Phrase'], phrase_counts_df['Count'])],
                    multi = True,
                    placeholder = 'Select keywords',
                    value = [],
                    style = {'width': '100%'}
                ),
                dbc.Button(
                    id = 'submit-button',
                    n_clicks = 0,
                    children = 'Submit',
                    color = 'primary',
                    style = {'margin-top': '10px'}
                ),
            ], width = {'size': '10'},
                style = {'font-size': '16px'}),

            dbc.Col([
                html.H2("Toggle Stack", style={'font-size': '20px'}),
                dcc.Checklist(
                    id = 'toggle-checklist',
                    options = [
                        {'label': 'Grouped', 'value': 'group'},
                    ], value = ['True'],
                    style = {'font-size': '16px'},
                )
            ])
        ], justify = 'between'
    ),

    dbc.Row([
        dbc.Col([
            dcc.Graph(id='keyword-graph')  # Populate graph based on the dropdown selection
        ]),
    ]),
])

#set up an callback function that updates the output based on our input
@app.callback(
    Output(component_id = 'keyword-graph', component_property = 'figure'),
    [Input(component_id = 'submit-button', component_property = 'n_clicks'),
     Input(component_id = 'toggle-checklist', component_property = 'value')],
    [dash.dependencies.State('keyword-dropdown', 'value')]
)

#create function that autosuggests words/phrases?
#def word_suggestor

#create function that updates figure
def update_graph(n_clicks, toggle, keywords):  
    #want it so that for each click, updates using property of n_clicks and changing the keywords input

    #consider the case that exist empty string
    if not keywords:
        #return empty dict if exist empty string
        return {}

    #apply the function here which applies strip() and lower() while splitting by ','
    df['keyword_frequency'] = df['combined_text'].apply(lambda x: count_keywords(x, keywords))

    #convert back to dataframe and fill zero if missing(N/A)
    keyword_df = df['keyword_frequency'].apply(pd.Series).fillna(0)

    #group by date and reset the index like before
    keyword_bydate_df = df[['Complete Loc Dt']].join(keyword_df).groupby('Complete Loc Dt').sum().reset_index()
    keyword_bydate_df = keyword_bydate_df.melt(id_vars = ['Complete Loc Dt'], var_name = 'Keyword', value_name = 'Service Calls')
    keyword_bydate_df = keyword_bydate_df.rename(columns = {'Complete Loc Dt': 'Date'})


    #set the barmode based on the toggle value
    barmode = 'group' if 'group' in toggle else 'stack'

    #create the histogram here
    fig = px.histogram(keyword_bydate_df, x = 'Date', y = 'Service Calls', color = 'Keyword', barmode = barmode, title = 'Phrase Presence Over Time')
    fig.update_xaxes(
        dtick=86400000.0 * 14 , #biweekly
        #tickformat="%b\n%Y",
        ticklabelmode="period"
    )

    #return the fig
    return fig
    
    
#############################

if __name__ == '__main__':
    app.run_server(debug = True, port = 8051) #specify port, couldn't terminate port 8050 so work around for now
    

In [20]:
import pandas as pd
import plotly.express as px
from plotly import graph_objects as go
import ipywidgets as widgets
from IPython.display import display

import numpy as np
import spacy
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

import dash
from dash import dcc, html
from dash.dependencies import Input, Output

import dash_bootstrap_components as dbc #used for formatting layout of the dash

nlp = spacy.load("en_core_web_sm")
nlp.max_length = 10000000

#File path to your Excel file
file_path = 'C:/Users/rgae/OneDrive - QuidelOrtho/Documents/All QuidelFiles/Excel Files/6-27-2024 2nd Copy .xlsx'
#Read the Excel file and extract necessary columns
df = pd.read_excel(file_path, index_col = None, na_values = ['NA'], usecols = 'AP, AV')

#use pd to date time to utilize in the mapping of the data over time
df['Incident Close Loc Dt'] = pd.to_datetime(df['Incident Close Loc Dt'])
#Combine the text columns into a single text column for analysis
df['combined_text'] = df.astype(str).agg(' '.join, axis=1)

#Tokenize the text using spaCy and create a list of sentences
def tokenize(text):
    doc = nlp(text.lower())
    return ' '.join([token.text for token in doc]) #https://stackoverflow.com/questions/57187116/how-to-modify-spacy-tokens-doc-doc-tokens-with-pipeline-components-in-spacy

df['tokenized_text'] = df['combined_text'].apply(tokenize) #call the method above to the combined text column 
#cant do df['tokenized_text'] = tokenize(...) because you can't input series into a string parameter. Utilizing apply allows us to get around this and apply it for each row



#Use CountVectorizer to get common phrases
vectorizer = CountVectorizer(ngram_range = (1, 4))  #Bigrams and trigrams (how long each phrase can be i.e. 2 and 3)
X = vectorizer.fit_transform(df['tokenized_text'])

#Sum up the counts of each phrase
phrase_counts = X.sum(axis=0).A1
phrases = vectorizer.get_feature_names_out()

#Create a DataFrame with phrases and their counts
phrase_counts_df = pd.DataFrame({'Phrase': phrases, 'Count': phrase_counts}).sort_values(by = 'Count', ascending = False).reset_index()
phrase_counts_df = phrase_counts_df.drop(['index'], axis = 1)

words_to_filter = ['the', 'be', 'to', 'of', 'and', 'a', 'in', 'that', 'I', 'i']
phrase_counts_df = phrase_counts_df[~phrase_counts_df['Phrase'].isin(words_to_filter)]
phrase_counts_df = phrase_counts_df.drop(phrase_counts_df[phrase_counts_df['Count'] <= 1].index)


#Define the function to count keyword frequencies
def count_keywords(text, keywords):
    text = text.lower()
    return {keyword: 1 if keyword in text else 0 for keyword in keywords} #binary values
#############################

#initialize app
app = dash.Dash(__name__, external_stylesheets = [dbc.themes.CYBORG]) 
app.layout = dbc.Container([ #dbc rows and col where number of columns and rows are determined by how many rows and columns are in the parameters
    dbc.Row([ #e.g. row(col col col) <-- 3 columns || row (col) <-- 1 column https://dash-bootstrap-components.opensource.faculty.ai/docs/components/layout/
        dbc.Col([
            html.H3("Phrase Analysis"), #H2 indicates sub heading with the following properties
            dcc.Dropdown(
                id='keyword-dropdown',
                options = [{'label': f"{phrase} ----- {freq}", 'value': phrase} for phrase, freq in zip(phrase_counts_df['Phrase'], phrase_counts_df['Count'])],
                multi=True,
                placeholder='Select keywords',
                value=[],
                style={'width': '100%'}
            ),
            dbc.Button(
                id='submit-button',
                n_clicks=0,
                children='Submit',
                color='primary',
                style={'margin-top': '10px'}
            ),
        ], width='6'),
        dbc.Col([

        ]),
    ], justify='left', style={'margin-top': '20px'}),
    dbc.Row([
        dbc.Col([
            dcc.Graph(id='keyword-graph')  # Populate graph based on the dropdown selection
        ]),
    ]),
])

#set up an callback function that updates the output based on our input
@app.callback(
    Output(component_id = 'keyword-graph', component_property = 'figure'),
    [Input(component_id = 'submit-button', component_property = 'n_clicks')],
    [dash.dependencies.State('keyword-dropdown', 'value')]
)

#create function that autosuggests words/phrases?
#def word_suggestor

#create function that updates figure
def update_graph(n_clicks, keywords):  
    #want it so that for each click, updates using property of n_clicks and changing the keywords input

    #consider the case that exist empty string
    if not keywords:
        #return empty dict if exist empty string
        return {}

    #apply the function here which applies strip() and lower() while splitting by ','
    df['keyword_frequency'] = df['combined_text'].apply(lambda x: count_keywords(x, keywords))

    #convert back to dataframe and fill zero if missing(N/A)
    keyword_df = df['keyword_frequency'].apply(pd.Series).fillna(0)

    #group by date and reset the index like before
    keyword_bydate_df = df[['Incident Close Loc Dt']].join(keyword_df).groupby('Incident Close Loc Dt').sum().reset_index()
    keyword_bydate_df = keyword_bydate_df.melt(id_vars = ['Incident Close Loc Dt'], var_name = 'Keyword', value_name = 'Service Calls')
    keyword_bydate_df = keyword_bydate_df.rename(columns = {'Incident Close Loc Dt': 'Date'})

    

    #create the histogram here
    fig = px.histogram(keyword_bydate_df, x = 'Date', y = 'Service Calls', color = 'Keyword', barmode = 'group', title = 'Phrase Presence Over Time') #toggle button, stack of not stack (BARMODE = VARIABLE)
    fig.update_xaxes(
        dtick=86400000.0 * 14 , #biweekly
        #tickformat="%b\n%Y",
        ticklabelmode="period"
    )

    #return the fig
    return fig
    
    
#############################

if __name__ == '__main__':
    app.run_server(debug = True, port = 8051) #specify port, couldn't terminate port 8050 so work around for now
    